In [1]:
from dgp import *
from estimation import *

# 1. Simulate or load data
adj = get_graph(2000, 1, 2, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=20,
    burnin_R=10, seed=0)

100%|██████████| 30/30 [00:06<00:00,  4.70it/s]


In [2]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=20,
    burnin_R=10,
    seed=0
)

{'average': 0.31014999999999987,
 'direct_effect': -0.13672499999999993,
 'spillover_effect': -0.054025000000000045,
 'psi_gamma': 0.31014999999999987,
 'psi_1_gamma': 0.270075,
 'psi_0_gamma': 0.40679999999999994,
 'psi_zero': 0.460825}

In [3]:
from drnet_new import doubly_robust

ret = {
    'average': [],
    'direct_effect': [],
    'spillover_effect': [],
    'psi_0_gamma': [],
    'psi_zero': []
}

for Y, A, L in tqdm(zip(Y_chain, A_chain, L_chain)):
    ret_i = doubly_robust(A, L, Y, adj, treatment_allocation=0.7, num_rep=1000)
    ret['average'].append(ret_i['average'])
    ret['direct_effect'].append(ret_i['direct_effect'])
    ret['spillover_effect'].append(ret_i['spillover_effect'])
    ret['psi_0_gamma'].append(ret_i['psi_0_gamma'])
    ret['psi_zero'].append(ret_i['psi_zero'])
    
print('Average effect:', np.mean(ret['average']))
print('Direct effect:', np.mean(ret['direct_effect']))
print('Spillover effect:', np.mean(ret['spillover_effect']))
print('Psi_0_gamma:', np.mean(ret['psi_0_gamma']))
print('Psi_zero:', np.mean(ret['psi_zero']))

1it [01:39, 99.02s/it]

[0.4618755154958654]


2it [03:19, 99.69s/it]

[0.43919043649913875]


3it [05:00, 100.57s/it]

...

[0.45985180122928904]


19it [37:18, 155.08s/it]

[0.45815532191679076]


20it [40:20, 121.05s/it]

[0.4636455078056823]
Average effect: 0.31985661462711434
Direct effect: -0.12977183855147084
Spillover effect: -0.03415350528899572
Psi_0_gamma: 0.40957343452756917
Psi_zero: 0.44372693981656486


In [3]:

from autognet import *

ret = {
    'average': [],
    'direct_effect': [],
    'spillover_effect': [],
    'psi_0_gamma': [],
    'psi_zero': []
}

for Y, A, L in tqdm(zip(Y_chain, A_chain, L_chain)):
    ret_i = evaluate_autog_effect(adj, Y, A, L, treatment_allocation=0.7, R=20, burnin=10, seed=1)
    ret['average'].append(ret_i['average'])
    ret['direct_effect'].append(ret_i['direct_effect'])
    ret['spillover_effect'].append(ret_i['spillover_effect'])
    ret['psi_0_gamma'].append(ret_i['psi_0_gamma'])
    ret['psi_zero'].append(ret_i['psi_zero'])
    
print('Average effect:', np.mean(ret['average']))
print('Direct effect:', np.mean(ret['direct_effect']))
print('Spillover effect:', np.mean(ret['spillover_effect']))
print('Psi_0_gamma:', np.mean(ret['psi_0_gamma']))
print('Psi_zero:', np.mean(ret['psi_zero']))

20it [21:14, 63.73s/it]

Average effect: 0.31758250000000005
Direct effect: -0.14854916666666665
Spillover effect: -0.02998833333333334
Psi_0_gamma: 0.4224724999999999
Psi_zero: 0.4524608333333333


In [4]:
from autognet2 import *

ret = {
    'average': [],
    'direct_effect': [],
    'spillover_effect': [],
    'psi_0_gamma': [],
    'psi_zero': []
}
for Y, A, L in tqdm(zip(Y_chain, A_chain, L_chain)):
    ret_i = evaluate_autognet_via_agc_effect(adj, Y, A, L, treatment_allocation=0.7, R=20, burnin=10, seed=1)
    ret['average'].append(ret_i['average'])
    ret['direct_effect'].append(ret_i['direct_effect'])
    ret['spillover_effect'].append(ret_i['spillover_effect'])
    ret['psi_0_gamma'].append(ret_i['psi_0_gamma'])
    ret['psi_zero'].append(ret_i['psi_zero'])
    
print('Average effect:', np.mean(ret['average']))
print('Direct effect:', np.mean(ret['direct_effect']))
print('Spillover effect:', np.mean(ret['spillover_effect']))
print('Psi_0_gamma:', np.mean(ret['psi_0_gamma']))
print('Psi_zero:', np.mean(ret['psi_zero']))

0it [00:00, ?it/s]

tau: [-1.13645939  0.1499439  -0.48768631]
rho: [[0.         0.20580144 0.24871142]
 [0.20580144 0.         0.24871142]
 [0.20580144 0.24871142 0.        ]]
nu: [[-0.03550432  0.10759982  0.08512062]
 [-0.03550432  0.10759982  0.08512062]
 [-0.03550432  0.10759982  0.08512062]]
beta: [-2.52707434e-01 -5.95056734e-01 -2.36716855e-01 -1.66189721e-01
 -9.68971856e-05 -1.95701585e-01 -4.84795630e-02  4.09998676e-01
  9.46219918e-02  1.83491841e-01]


1it [00:29, 29.32s/it]

tau: [-1.1107394   0.3986641  -0.60325825]
rho: [[0.         0.21174625 0.13475569]
 [0.21174625 0.         0.13475569]
 [0.21174625 0.13475569 0.        ]]
nu: [[ 0.21475473  0.07905419 -0.07523203]
 [ 0.21475473  0.07905419 -0.07523203]
 [ 0.21475473  0.07905419 -0.07523203]]
beta: [-0.43742518 -0.61006846 -0.08721581 -0.14607116  0.03676814 -0.14114223
  0.12712199  0.30808405 -0.09612312  0.18543277]


2it [00:59, 29.92s/it]

...

tau: [-1.19517486  0.47771422 -0.59222865]
rho: [[0.         0.01308826 0.25619686]
 [0.01308826 0.         0.25619686]
 [0.01308826 0.25619686 0.        ]]
nu: [[0.20273193 0.11359214 0.06406044]
 [0.20273193 0.11359214 0.06406044]
 [0.20273193 0.11359214 0.06406044]]
beta: [-0.51579402 -0.55072113 -0.18187963 -0.18923376 -0.04553936 -0.03925427
  0.05892806  0.40585178  0.03493164  0.27002821]


19it [08:56, 26.86s/it]

tau: [-1.2303566   0.52104389 -0.69517491]
rho: [[0.         0.13086814 0.43503132]
 [0.13086814 0.         0.43503132]
 [0.13086814 0.43503132 0.        ]]
nu: [[0.05082769 0.14024861 0.01786352]
 [0.05082769 0.14024861 0.01786352]
 [0.05082769 0.14024861 0.01786352]]
beta: [-0.21105369 -0.48647114 -0.21121919 -0.18767702  0.00431629 -0.14375521
 -0.07608749  0.24348686  0.03797799  0.1848244 ]


20it [09:23, 28.17s/it]

Average effect: 0.3201225
Direct effect: -0.14929499999999996
Spillover effect: -0.03713875000000002
Psi_0_gamma: 0.4223162499999999
Psi_zero: 0.459455


In [5]:
from dgp import *
from estimation import *

# 1. Simulate or load data
adj = get_graph(2000, 8, 10, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=20,
    burnin_R=10, seed=0)

100%|██████████| 30/30 [00:06<00:00,  4.83it/s]


In [6]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=20,
    burnin_R=10,
    seed=0
)

{'average': 0.14565,
 'direct_effect': -0.10649999999999994,
 'spillover_effect': -0.45730000000000004,
 'psi_gamma': 0.14565,
 'psi_1_gamma': 0.1165,
 'psi_0_gamma': 0.22299999999999995,
 'psi_zero': 0.6803}

In [7]:

from autognet import *

ret = {
    'average': [],
    'direct_effect': [],
    'spillover_effect': [],
    'psi_0_gamma': [],
    'psi_zero': []
}

for Y, A, L in tqdm(zip(Y_chain, A_chain, L_chain)):
    ret_i = evaluate_autog_effect(adj, Y, A, L, treatment_allocation=0.7, R=20, burnin=10, seed=1)
    ret['average'].append(ret_i['average'])
    ret['direct_effect'].append(ret_i['direct_effect'])
    ret['spillover_effect'].append(ret_i['spillover_effect'])
    ret['psi_0_gamma'].append(ret_i['psi_0_gamma'])
    ret['psi_zero'].append(ret_i['psi_zero'])
    
print('Average effect:', np.mean(ret['average']))
print('Direct effect:', np.mean(ret['direct_effect']))
print('Spillover effect:', np.mean(ret['spillover_effect']))
print('Psi_0_gamma:', np.mean(ret['psi_0_gamma']))
print('Psi_zero:', np.mean(ret['psi_zero']))

20it [22:04, 66.21s/it]

Average effect: 0.134595
Direct effect: -0.11127416666666667
Spillover effect: -0.305315
Psi_0_gamma: 0.2177616666666667
Psi_zero: 0.5230766666666666


In [8]:
from autognet2 import *

ret = {
    'average': [],
    'direct_effect': [],
    'spillover_effect': [],
    'psi_0_gamma': [],
    'psi_zero': []
}
for Y, A, L in tqdm(zip(Y_chain, A_chain, L_chain)):
    ret_i = evaluate_autognet_via_agc_effect(adj, Y, A, L, treatment_allocation=0.7, R=20, burnin=10, seed=1)
    ret['average'].append(ret_i['average'])
    ret['direct_effect'].append(ret_i['direct_effect'])
    ret['spillover_effect'].append(ret_i['spillover_effect'])
    ret['psi_0_gamma'].append(ret_i['psi_0_gamma'])
    ret['psi_zero'].append(ret_i['psi_zero'])
    
print('Average effect:', np.mean(ret['average']))
print('Direct effect:', np.mean(ret['direct_effect']))
print('Spillover effect:', np.mean(ret['spillover_effect']))
print('Psi_0_gamma:', np.mean(ret['psi_0_gamma']))
print('Psi_zero:', np.mean(ret['psi_zero']))

0it [00:00, ?it/s]

tau: [-0.87527741  0.13626941 -0.39778239]
rho: [[0.         0.24114617 0.14185871]
 [0.24114617 0.         0.14185871]
 [0.24114617 0.14185871 0.        ]]
nu: [[ 0.04038593 -0.01618387  0.02939917]
 [ 0.04038593 -0.01618387  0.02939917]
 [ 0.04038593 -0.01618387  0.02939917]]
beta: [-1.71910925 -0.67286188 -0.02308216 -0.36835282  0.02785888 -0.12095023
 -0.07773765  0.57536678  0.01307376  0.35426729]


1it [00:26, 26.44s/it]

tau: [-1.64220904  0.55302803 -0.36476552]
rho: [[0.         0.08981069 0.1470832 ]
 [0.08981069 0.         0.1470832 ]
 [0.08981069 0.1470832  0.        ]]
nu: [[0.05889033 0.09774563 0.04146942]
 [0.05889033 0.09774563 0.04146942]
 [0.05889033 0.09774563 0.04146942]]
beta: [-5.99034275e-01 -3.72955969e-01 -1.10653481e-01 -1.49175504e-01
 -8.76284264e-02 -3.21102531e-01 -1.75480386e-02  3.23189964e-01
 -5.71968243e-05  2.08602056e-02]


2it [00:53, 26.71s/it]

...

tau: [-1.39108076  0.9054155  -0.3212202 ]
rho: [[0.         0.11749791 0.21435826]
 [0.11749791 0.         0.21435826]
 [0.11749791 0.21435826 0.        ]]
nu: [[0.11928996 0.01464712 0.04379251]
 [0.11928996 0.01464712 0.04379251]
 [0.11928996 0.01464712 0.04379251]]
beta: [-0.07011632 -0.67614031 -0.22765947 -0.19550315 -0.03241603  0.34521927
 -0.09445323  0.34777399  0.03055813  0.33182298]


19it [10:06, 36.70s/it]

tau: [-1.24568896  0.56517146 -0.4213584 ]
rho: [[0.         0.11820225 0.23751782]
 [0.11820225 0.         0.23751782]
 [0.11820225 0.23751782 0.        ]]
nu: [[ 0.15624419 -0.01031722  0.01915192]
 [ 0.15624419 -0.01031722  0.01915192]
 [ 0.15624419 -0.01031722  0.01915192]]
beta: [-0.1428414  -0.75583755 -0.21465294 -0.23448078  0.00136948  0.06081438
  0.00223419  0.38379044 -0.06318158  0.23362598]


20it [10:32, 31.62s/it]

Average effect: 0.1400975
Direct effect: -0.11814875
Spillover effect: -0.3076212499999999
Psi_0_gamma: 0.22807999999999998
Psi_zero: 0.53570125
